In [13]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

In [14]:
train_path="F:/Data/Data/classification/train_val/train"
test_path="F:/Data/Data/classification/test"
val_path="F:/Data/Data/classification/train_val/val"

In [15]:
x_train=[]

for folder in os.listdir(train_path):

    sub_path=train_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(250,250))

        x_train.append(img_arr)

x_test=[]

for folder in os.listdir(test_path):

    sub_path=test_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(250,250))

        x_test.append(img_arr)

x_val=[]

for folder in os.listdir(val_path):

    sub_path=val_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(250,250))

        x_val.append(img_arr)

In [16]:
train_x=np.array(x_train)
test_x=np.array(x_test)
val_x=np.array(x_val)

In [17]:
train_datagen = ImageDataGenerator(rotation_range=45)
test_datagen = ImageDataGenerator(rotation_range=45)
val_datagen = ImageDataGenerator(rotation_range=45)

In [18]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (250,250),
                                                 batch_size = 8,
                                                 class_mode = 'sparse')
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (250,250),
                                            batch_size = 8,
                                            class_mode = 'sparse')
val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (250,250),
                                            batch_size = 8,
                                            class_mode = 'sparse')

Found 1344 images belonging to 4 classes.
Found 416 images belonging to 4 classes.
Found 332 images belonging to 4 classes.


In [19]:
train_y=training_set.classes
test_y=test_set.classes
val_y=val_set.classes

In [20]:
training_set.class_indices

{'NoSeatBelt': 0, 'NoSeatBelt_Phone': 1, 'SeatBelt': 2, 'SeatBelt_Phone': 3}

In [38]:
train_y.shape,test_y.shape,val_y.shape

((1344,), (416,), (332,))

In [39]:
from tensorflow.keras.applications import ResNet50

IMAGE_SIZE = [250,250]
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
#vgg = ResNet50(input_shape=(224,224,3), include_top=False, weights="imagenet")

In [40]:
#do not train the pre-trained layers of VGG-19
for layer in vgg.layers:
    layer.trainable = False

In [41]:
x = Flatten()(vgg.output)

In [42]:
#adding output layer.Softmax classifier is used as it is multi-class classification
prediction = Dense(4, activation='softmax')(x)

model = Model(inputs=vgg.input, outputs=prediction)

In [43]:
# view the structure of the model
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 250, 250, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 250, 250, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 250, 250, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 125, 125, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 125, 125, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 125, 125, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 62, 62, 128)       0   

In [44]:
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

In [45]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)
#Early stopping to avoid overfitting of model

In [47]:
# fit the model
history = model.fit(
  train_x,
  train_y,
  validation_data=(val_x,val_y),
  epochs=40,
  batch_size=8,shuffle=True)

Epoch 1/40
168/168 [==============================] - 10s 61ms/step - loss: 0.8203 - accuracy: 0.9754 - val_loss: 14.6150 - val_accuracy: 0.8916
Epoch 2/40
168/168 [==============================] - 10s 60ms/step - loss: 1.5830 - accuracy: 0.9702 - val_loss: 10.3622 - val_accuracy: 0.8855
Epoch 3/40
168/168 [==============================] - 10s 60ms/step - loss: 1.7833 - accuracy: 0.9643 - val_loss: 13.8284 - val_accuracy: 0.8494
Epoch 4/40
168/168 [==============================] - 10s 60ms/step - loss: 1.6423 - accuracy: 0.9650 - val_loss: 13.4623 - val_accuracy: 0.9036
Epoch 5/40
168/168 [==============================] - 10s 60ms/step - loss: 1.8349 - accuracy: 0.9710 - val_loss: 14.4912 - val_accuracy: 0.8494
Epoch 6/40
168/168 [==============================] - 10s 60ms/step - loss: 0.7164 - accuracy: 0.9821 - val_loss: 12.5607 - val_accuracy: 0.8976
Epoch 7/40
168/168 [==============================] - 10s 60ms/step - loss: 0.6176 - accuracy: 0.9844 - val_loss: 17.2721 - val_ac

In [48]:
model.evaluate (test_x, test_y)

13/13 [==============================] - 6s 158ms/step - loss: 10.9045 - accuracy: 0.9087


[10.904470443725586, 0.9086538553237915]

In [49]:
model.save('model.h5')